In [3]:
import quandl
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import SGDClassifier
from sklearn import preprocessing, cross_validation
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
quandl.ApiConfig.api_key = 'DEN8cmx6MYrtzyeUdDeh'

In [5]:
%%time
tickers = ['FB','GOOGL','AAPl','AMZN','TSLA','MSFT','AMD','INTC','T','NFLX']
Validate = []
Train = []
for ticker in tickers:
    df = quandl.get("WIKI/"+ticker)
    df = df.iloc[-500:]
    func = ['mean','std','skew','kurt']
    time = [2,4,7,30,60,120,250]
    Z = pd.DataFrame()
    for f in func:
        for t in time:
            aux = pd.DataFrame()
            for i in range(249):
                aux[df.index[i+251]] = list(df.iloc[i-t+251:i+251]['Adj. Close'])
            Z[f+str(t)] = aux.apply(f,axis=0)  
    Z.drop(['skew2','kurt2'],axis=1, inplace =True)
    W = pd.merge(Z, df[['Adj. Close']],left_index=True,right_index=True)
    X = W.drop(['Adj. Close'],1).fillna(0)
    y = W[['Adj. Close']]
    mm1 = MinMaxScaler()
    mm2 = MinMaxScaler()
    mm1.fit(X)
    mm2.fit(y)
    Xm = pd.DataFrame(mm1.transform(X))
    ym = pd.DataFrame(mm2.transform(y))
    Xt, Xv, yt, yv = cross_validation.train_test_split(Xm, ym, test_size = 0.3)
    model =  KNeighborsRegressor()
    param_grid = dict(n_neighbors  = range(20,70),
                      weights = ['uniform','distance'],
                      algorithm=['auto', 'ball_tree', 'kd_tree', 'brute']) 
    grid = GridSearchCV(cv = 3, 
                        estimator = model,
                        n_jobs = -1,
                        param_grid=param_grid)
    grid.fit(Xm,ym)
    model = grid.best_estimator_
    model.fit(Xt,yt)
    print ticker
    print 'ACC Validate {:.2%}'.format(model.score(Xv,yv))
    print 'ACC Train    {:.2%}'.format(model.score(Xt,yt))
    Validate.append(model.score(Xv,yv))
    Train.append(model.score(Xt,yt))
print 'Promedio Validate  {:.2%}'.format(np.array(Validate).mean())
print 'Promedio Train     {:.2%}'.format(np.array(Train).mean())

FB
ACC Validate 88.15%
ACC Train    100.00%
GOOGL
ACC Validate 87.78%
ACC Train    100.00%
AAPl
ACC Validate 91.40%
ACC Train    100.00%
AMZN
ACC Validate 97.82%
ACC Train    100.00%
TSLA
ACC Validate 66.79%
ACC Train    100.00%
MSFT
ACC Validate 95.46%
ACC Train    100.00%
AMD
ACC Validate 62.19%
ACC Train    100.00%
INTC
ACC Validate 95.81%
ACC Train    100.00%
T
ACC Validate 38.30%
ACC Train    41.22%
NFLX
ACC Validate 96.40%
ACC Train    100.00%
Promedio Validate  82.01%
Promedio Train     94.12%
CPU times: user 2min 5s, sys: 3.78 s, total: 2min 8s
Wall time: 3min 15s
